In [1]:
import pandas as pd

In [148]:
df = pd.read_csv("raw/table-71.csv").fillna(-1).convert_dtypes().rename(columns={
  'opened_ym': 'date',
  'control_fi': 'isControlled',
  'loan_type': 'loanType',
  'grp_age': 'ageGroup',
  'fi_type': 'fiType',
  'flag_urban_2021': 'isUrban',
  'region_code': 'region',
  'flag_newbrw': 'isNewBorrower',
  'sum_curbal': 'amount',
  'num_ac': 'accounts'
})

# use 0 for NaNs in value columns
df['amount'] = df['amount'].replace(-1, 0)
df['accounts'] = df['accounts'].replace(-1, 0)

unique = ['date', 'isControlled']
facets = list(df.columns[2:8])
values = ['amount', 'accounts']
df = df.sort_values(unique)[unique + facets + values]

In [218]:
# one way to do it is split the database into many json files

for f in facets:
  x = unique + [f]
  tmp = df[x + values].groupby(x, dropna=False).sum()
  new_index = pd.MultiIndex.from_product([tmp.index.unique(level=0), tmp.index.unique(level=1), tmp.index.unique(level=2)])
  tmp = tmp.reindex(new_index).fillna(0)
  tmp = tmp.sort_values([f] + unique[::-1]).droplevel(0)
  groupby = [f] + unique[1:]
  tmp = tmp.groupby(groupby).agg(
    amount=('amount', pd.Series.to_list),
    accounts=('accounts', pd.Series.to_list)
  )
  tmp = tmp.reset_index()
  tmp[groupby + ['amount']].to_json('json/table-71-' + f + '-amount.json', orient="records")
  tmp[groupby + ['accounts']].to_json('json/table-71-' + f + '-accounts.json', orient="records")